In [4]:
import os

# List of files
files = [
    "HYP-LONGEND00_axis.npy",
    "HYP-LONGEND00_CL_SEM.npy",
    "HYP-LONGEND01_axis.npy",
    "HYP-LONGEND01_CL_SEM.npy",
    "HYP-LONGEND02_axis.npy",
    "HYP-LONGEND02_CL_SEM.npy",
    "HYP-LONGEND03_axis.npy",
    "HYP-LONGEND03_CL_SEM.npy",
    "HYP-LONGEND04_axis.npy",
    "HYP-LONGEND04_CL_SEM.npy",
    "HYP-LONGEND05_axis.npy",
    "HYP-LONGEND05_CL_SEM.npy",
    "HYP-LONGEND06_axis.npy",
    "HYP-LONGEND06_CL_SEM.npy",
    "HYP-LONGEND-06-REDO_axis.npy",
    "HYP-LONGEND-06-REDO_CL_SEM.npy",
    "HYP-LONGEND07_axis.npy",
    "HYP-LONGEND07_CL_SEM.npy",
    "HYP-LONGEND-07-REDO_axis.npy",
    "HYP-LONGEND-07-REDO_CL_SEM.npy",
    "HYP-LONGEND08_axis.npy",
    "HYP-LONGEND08_CL_SEM.npy",
    "HYP-LONGEND-08-REDO_axis.npy",
    "HYP-LONGEND-08-REDO_CL_SEM.npy",
    "HYP-LONGEND09_axis.npy",
    "HYP-LONGEND09_CL_SEM.npy",
    "HYP-SHORTEND-01_axis.npy",
    "HYP-SHORTEND-01_CL_SEM.npy",
    "HYP-SHORTEND-02_axis.npy",
    "HYP-SHORTEND-02_CL_SEM.npy",
    "HYP-SHORTEND-03_axis.npy",
    "HYP-SHORTEND-03_CL_SEM.npy",
    "HYP-SHORTEND-04_axis.npy",
    "HYP-SHORTEND-04_CL_SEM.npy",
    "HYP-SHORTEND-04-REDO2_axis.npy",
    "HYP-SHORTEND-04-REDO2_CL_SEM.npy",
    "HYP-SHORTEND-05_axis.npy",
    "HYP-SHORTEND-05_CL_SEM.npy",
    "HYP-SHORTEND-05-REDO_axis.npy",
    "HYP-SHORTEND-05-REDO_CL_SEM.npy",
    "HYP-SHORTEND-06_axis.npy",
    "HYP-SHORTEND-06_CL_SEM.npy",
    "HYP-SHORTEND-07_axis.npy",
    "HYP-SHORTEND-07_CL_SEM.npy",
    "HYP-SHORTEND07-REDO700_axis.npy",
    "HYP-SHORTEND07-REDO700_CL_SEM.npy",
    "HYP-SHORTEND-08_axis.npy",
    "HYP-SHORTEND-08_CL_SEM.npy",
    "HYP-SHORTEND08-REDO7000_axis.npy",
    "HYP-SHORTEND08-REDO7000_CL_SEM.npy",
    "HYP-SHORTEND08-REDO700_axis.npy",
    "HYP-SHORTEND08-REDO700_CL_SEM.npy",
    "HYP-SHORTEND-09_axis.npy",
    "HYP-SHORTEND-09_CL_SEM.npy",
    "HYP-SHORTEND09-REDO700-2_axis.npy",
    "HYP-SHORTEND09-REDO700-2_CL_SEM.npy",
    "HYP-SHORTEND-10_axis.npy",
    "HYP-SHORTEND-10_CL_SEM.npy",
    "HYP-SHORTEND10-REDO700_axis.npy",
    "HYP-SHORTEND10-REDO700_CL_SEM.npy"
]

# Root path
root_path = "/rds/user/cja69/hpc-work/python/cl_data"

# Base template
template = """if __name__ == "__main__":
    import multiprocessing
    multiprocessing.set_start_method('spawn', force=True)
    import numpy as np
    from datetime import datetime
    from scipy.optimize import curve_fit
    from concurrent.futures import ThreadPoolExecutor
    from tqdm import tqdm

    filepath = "{root}/{cl_file}"
    axis_value_file = "{root}/{axis_file}"
    output_log = "{root}/{basename}_cl_fit.log"
    output_path = "{root}/{basename}"

    with open(output_log, 'w') as log_file:
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"CL fit script started at {{current_time}}\\n")

    def gaussian(x, center, fwhm, height, offset):
        sigma = fwhm / (2 * np.sqrt(2 * np.log(2)))
        return height * np.exp(-(x - center)**2 / (2 * sigma**2)) + offset

    # shape (256, 256, 1024)
    image = np.load(filepath)
    axis_values = np.load(axis_value_file)

    pixels = image.reshape(-1, image.shape[-1])

    center_guess = 3.4
    fwhm_guess = 0.1
    height_guess = 500
    offset_guess = 200

    lower_bounds = [center_guess - 0.2, 0.01, 0, 0]
    upper_bounds = [center_guess + 0.2, 1.0, 1e5, 1e4]
    initial_guess = [center_guess, fwhm_guess, height_guess, offset_guess]

    def fit_spectrum(spectrum):
        try:
            popt, _ = curve_fit(
                gaussian,
                axis_values,
                spectrum,
                p0=initial_guess,
                bounds=(lower_bounds, upper_bounds),
                maxfev=500
            )
        except RuntimeError:
            popt = [np.nan, np.nan, np.nan, np.nan]
        return popt

    results = []
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(fit_spectrum, pixels), total=pixels.shape[0]))

    param_maps = np.array(results).reshape(image.shape[0], image.shape[1], 4)

    np.save(output_path + '_m_centre.npy', param_maps[:, :, 0])
    np.save(output_path + '_m_fwhm.npy', param_maps[:, :, 1])
    np.save(output_path + '_m_intensity.npy', param_maps[:, :, 2])

    with open(output_log, 'a') as log_file:
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"CL fit script finished at {{current_time}}\\n")
"""

# Pair up axis and CL_SEM files
pairs = {}
for f in files:
    if f.endswith("_axis.npy"):
        key = f.replace("_axis.npy", "")
        pairs.setdefault(key, {})["axis"] = f
    elif f.endswith("_CL_SEM.npy"):
        key = f.replace("_CL_SEM.npy", "")
        pairs.setdefault(key, {})["cl"] = f

# Generate one Python script per dataset
outdir = "generated_scripts"
os.makedirs(outdir, exist_ok=True)

for key, filepair in pairs.items():
    if "axis" in filepair and "cl" in filepair:
        script_content = template.format(
            root=root_path,
            axis_file=filepair["axis"],
            cl_file=filepair["cl"],
            basename=key
        )
        script_path = os.path.join(outdir, f"{key}_fit.py")
        with open(script_path, "w") as f:
            f.write(script_content)
        print(f"Generated {script_path}")


Generated generated_scripts\HYP-LONGEND00_fit.py
Generated generated_scripts\HYP-LONGEND01_fit.py
Generated generated_scripts\HYP-LONGEND02_fit.py
Generated generated_scripts\HYP-LONGEND03_fit.py
Generated generated_scripts\HYP-LONGEND04_fit.py
Generated generated_scripts\HYP-LONGEND05_fit.py
Generated generated_scripts\HYP-LONGEND06_fit.py
Generated generated_scripts\HYP-LONGEND-06-REDO_fit.py
Generated generated_scripts\HYP-LONGEND07_fit.py
Generated generated_scripts\HYP-LONGEND-07-REDO_fit.py
Generated generated_scripts\HYP-LONGEND08_fit.py
Generated generated_scripts\HYP-LONGEND-08-REDO_fit.py
Generated generated_scripts\HYP-LONGEND09_fit.py
Generated generated_scripts\HYP-SHORTEND-01_fit.py
Generated generated_scripts\HYP-SHORTEND-02_fit.py
Generated generated_scripts\HYP-SHORTEND-03_fit.py
Generated generated_scripts\HYP-SHORTEND-04_fit.py
Generated generated_scripts\HYP-SHORTEND-04-REDO2_fit.py
Generated generated_scripts\HYP-SHORTEND-05_fit.py
Generated generated_scripts\HYP-S

In [7]:
import os

# List of dataset base names (matching the fit scripts we generated earlier)
datasets = [
    "HYP-LONGEND00",
    "HYP-LONGEND01",
    "HYP-LONGEND02",
    "HYP-LONGEND03",
    "HYP-LONGEND04",
    "HYP-LONGEND05",
    "HYP-LONGEND06",
    "HYP-LONGEND-06-REDO",
    "HYP-LONGEND07",
    "HYP-LONGEND-07-REDO",
    "HYP-LONGEND08",
    "HYP-LONGEND-08-REDO",
    "HYP-LONGEND09",
    "HYP-SHORTEND-01",
    "HYP-SHORTEND-02",
    "HYP-SHORTEND-03",
    "HYP-SHORTEND-04",
    "HYP-SHORTEND-04-REDO2",
    "HYP-SHORTEND-05",
    "HYP-SHORTEND-05-REDO",
    "HYP-SHORTEND-06",
    "HYP-SHORTEND-07",
    "HYP-SHORTEND07-REDO700",
    "HYP-SHORTEND-08",
    "HYP-SHORTEND08-REDO7000",
    "HYP-SHORTEND08-REDO700",
    "HYP-SHORTEND-09",
    "HYP-SHORTEND09-REDO700-2",
    "HYP-SHORTEND-10",
    "HYP-SHORTEND10-REDO700",
]

# Output directory for submission scripts
outdir = "submission_scripts"
os.makedirs(outdir, exist_ok=True)

# Template for SLURM submission script
template = """#!/bin/bash
#SBATCH --account KUSCH-SL3-CPU
#SBATCH --partition cclake
#SBATCH --nodes 1
#SBATCH --ntasks-per-node 1
#SBATCH --cpus-per-task 1
#SBATCH --time 00:30:00
#SBATCH -o {name}_model_fit.out
#SBATCH -e {name}_model_fit.err

# Module setup: cluster environment and recent python.
module purge
module load rhel8/default-ccl
module load python/3.11.9/gcc/nptrdpll

# Activate relevant python virtual environment.
source /rds/user/cja69/hpc-work/python/lumispy/bin/activate

# Let PyTorch use all available CPU cores for OpenMP, MKL and MKL-DNN backends
export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK
export MKL_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Install required Python packages
pip install --user numpy scipy

# Run the generated Python script
python /rds/user/cja69/hpc-work/python/generated_scripts/{name}_fit.py
"""

# Generate submission scripts with UNIX line breaks
for name in datasets:
    script_content = template.format(name=name)
    script_path = os.path.join(outdir, f"{name}_submit.sh")
    with open(script_path, "w", newline="\n") as f:  # enforce LF line endings
        f.write(script_content)
    print(f"Generated {script_path}")


Generated submission_scripts\HYP-LONGEND00_submit.sh
Generated submission_scripts\HYP-LONGEND01_submit.sh
Generated submission_scripts\HYP-LONGEND02_submit.sh
Generated submission_scripts\HYP-LONGEND03_submit.sh
Generated submission_scripts\HYP-LONGEND04_submit.sh
Generated submission_scripts\HYP-LONGEND05_submit.sh
Generated submission_scripts\HYP-LONGEND06_submit.sh
Generated submission_scripts\HYP-LONGEND-06-REDO_submit.sh
Generated submission_scripts\HYP-LONGEND07_submit.sh
Generated submission_scripts\HYP-LONGEND-07-REDO_submit.sh
Generated submission_scripts\HYP-LONGEND08_submit.sh
Generated submission_scripts\HYP-LONGEND-08-REDO_submit.sh
Generated submission_scripts\HYP-LONGEND09_submit.sh
Generated submission_scripts\HYP-SHORTEND-01_submit.sh
Generated submission_scripts\HYP-SHORTEND-02_submit.sh
Generated submission_scripts\HYP-SHORTEND-03_submit.sh
Generated submission_scripts\HYP-SHORTEND-04_submit.sh
Generated submission_scripts\HYP-SHORTEND-04-REDO2_submit.sh
Generated su

In [3]:
import os

datasets = [
    "HYP-LONGEND00",
    "HYP-LONGEND01",
    "HYP-LONGEND02",
    "HYP-LONGEND03",
    "HYP-LONGEND04",
    "HYP-LONGEND05",
    "HYP-LONGEND06",
    "HYP-LONGEND-06-REDO",
    "HYP-LONGEND07",
    "HYP-LONGEND-07-REDO",
    "HYP-LONGEND08",
    "HYP-LONGEND-08-REDO",
    "HYP-LONGEND09",
    "HYP-SHORTEND-01",
    "HYP-SHORTEND-02",
    "HYP-SHORTEND-03",
    "HYP-SHORTEND-04",
    "HYP-SHORTEND-04-REDO2",
    "HYP-SHORTEND-05",
    "HYP-SHORTEND-05-REDO",
    "HYP-SHORTEND-06",
    "HYP-SHORTEND-07",
    "HYP-SHORTEND07-REDO700",
    "HYP-SHORTEND-08",
    "HYP-SHORTEND08-REDO7000",
    "HYP-SHORTEND08-REDO700",
    "HYP-SHORTEND-09",
    "HYP-SHORTEND09-REDO700-2",
    "HYP-SHORTEND-10",
    "HYP-SHORTEND10-REDO700",
]

out_path = "submit_all.sh"
root_path = "/rds/user/cja69/hpc-work/python/submission_scripts"

with open(out_path, "w", newline="\n") as f:
    f.write("#!/bin/bash\n\n")
    for name in datasets:
        f.write(f"sbatch {root_path}/{name}_submit.sh\n")

print(f"Wrote {out_path}")


Wrote submit_all.sh


In [ ]:
afm_files = [
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_01.0_00000_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_01.0_00001_2.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_01.0_00002_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_01.0_00003_game_gunk.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_04_v2.0_00005_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_05.0_00000_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_06.0_00000_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_07.0_00000_game.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_08.0_00000_game_gunk.png',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\AFM\\25_06_2025\\Si_L_longend_09.0_00000_game.png'
]

cl_files = [
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND00\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND01\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND02\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND03\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND04\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND05\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND06\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND07\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND08\\HYPCard_corrected.hspy',
    'C:\\Users\\cobia\\OneDrive - University of Cambridge\\CL\\COBI20250707\\HYP-LONGEND09\\HYPCard_corrected.hspy'
]

In [13]:
import shutil

output_folder = r"C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\Alignment\CSVs"


for afm,cl,aln in zip(afm_files, cl_files, alignment_files):
    afm_base = os.path.splitext(os.path.basename(afm))[0]
    cl_base = cl.split(os.path.sep)[-2]

    target = os.path.join(output_folder, f"{cl_base}_{afm_base}.csv")

    print(f"Copying {aln} to {target}")

    shutil.copy2(aln, target)

Copying C:\Users\cobia\OneDrive - University of Cambridge\image_alignment\Si_L_longend_01.0_00000_game.csv to C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\Alignment\CSVs\HYP-LONGEND00_Si_L_longend_01.0_00000_game.csv
Copying C:\Users\cobia\OneDrive - University of Cambridge\image_alignment\Si_L_longend_01.0_00001_game_2.csv to C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\Alignment\CSVs\HYP-LONGEND01_Si_L_longend_01.0_00001_2.csv
Copying C:\Users\cobia\OneDrive - University of Cambridge\image_alignment\Si_L_longend_01.0_00002_game.csv to C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\Alignment\CSVs\HYP-LONGEND02_Si_L_longend_01.0_00002_game.csv
Copying C:\Users\cobia\OneDrive - University of Cambridge\image_alignment\Si_L_longend_01.0_00003_game_gunk.csv to C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\Alignment\CSVs\HYP-LONGEND03_Si_L_longend_01.0_00003_game_gunk.csv
Copying C:\Users\cobia\OneDrive - University of Cambrid

In [19]:
os.listdir(afm_dir).extend(["0","0","0"])

In [30]:
import pandas as pd

cl_dir = r"C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\CL\hyperspectral_cubes"
afm_dir = r"C:\Users\cobia\OneDrive - University of Cambridge\HF_Database\AFM\processed"

cl_files = [file for file in os.listdir(cl_dir) if file[-10:] == 'CL_SEM.npy']


afm_files = os.listdir(afm_dir)

In [32]:
afm_files

['fat_end_outside_L_03.0_00000.spm_corrected.npy',
 'fat_end_outside_L_04.0_00000.spm_corrected.npy',
 'fat_end_outside_L_05.0_00000.spm_corrected.npy',
 'fat_end_outside_L_06.0_00000.spm_corrected.npy',
 'fat_end_outside_L_07.0_00000.spm_corrected.npy',
 'fat_end_outside_L_08.0_00000.spm_corrected.npy',
 'fat_end_outside_L_09.0_00000.spm_corrected.npy',
 'fat_end_outside_L_10.0_00000.spm_corrected.npy',
 'short_end_01.0_00000.spm_corrected.npy',
 'short_end_02.0_00000.spm_corrected.npy',
 'short_end_03.0_00000.spm_corrected.npy',
 'short_end_04.0_00000.spm_corrected.npy',
 'short_end_05.0_00000.spm_corrected.npy',
 'short_end_06.0_00000.spm_corrected.npy',
 'short_end_07.0_00000.spm_corrected.npy',
 'short_end_08.0_00000.spm_corrected.npy',
 'short_end_09.0_00000.spm_corrected.npy',
 'short_end_10.0_00000.spm_corrected.npy',
 'Si_L_longend_01.0_00000_game.spm_corrected.npy',
 'Si_L_longend_01.0_00001_2.spm_corrected.npy',
 'Si_L_longend_01.0_00002_game.spm_corrected.npy',
 'Si_L_longe

In [33]:
['1','2',None,'3']

['1', '2', None, '3']

In [ ]:
cl_files
afm_files = [
    'Si_L_longend_06.0_00000_game.spm_corrected.npy',
    'Si_L_longend_07.0_00000_game.spm_corrected.npy',
    None,
    ''
]

['HYP-LONGEND-06-REDO_CL_SEM.npy',
 'HYP-LONGEND-07-REDO_CL_SEM.npy',
 'HYP-LONGEND-08-REDO_CL_SEM.npy',
 'HYP-LONGEND00_CL_SEM.npy',
 'HYP-LONGEND01_CL_SEM.npy',
 'HYP-LONGEND02_CL_SEM.npy',
 'HYP-LONGEND03_CL_SEM.npy',
 'HYP-LONGEND04_CL_SEM.npy',
 'HYP-LONGEND05_CL_SEM.npy',
 'HYP-LONGEND09_CL_SEM.npy',
 'HYP-SHORTEND-01_CL_SEM.npy',
 'HYP-SHORTEND-02_CL_SEM.npy',
 'HYP-SHORTEND-03_CL_SEM.npy',
 'HYP-SHORTEND-04-REDO2_CL_SEM.npy',
 'HYP-SHORTEND-05-REDO_CL_SEM.npy',
 'HYP-SHORTEND-06_CL_SEM.npy',
 'HYP-SHORTEND07-REDO700_CL_SEM.npy',
 'HYP-SHORTEND08-REDO700_CL_SEM.npy',
 'HYP-SHORTEND09-REDO700-2_CL_SEM.npy',
 'HYP-SHORTEND10-REDO700_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-03-AFM_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-04-AFM-REDO_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-05-AFM-REDO_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-06-AFM-REDO_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-07_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-08_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSIDE-09_CL_SEM.npy',
 'HYP-SI-FAT-END-OUTSID